<a href="https://colab.research.google.com/github/RedBatProject/ufo-works/blob/main/spell_correction_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import clear_output

In [2]:
!pip install datasets
clear_output()

In [3]:

from datasets import load_dataset
import pandas as pd
from collections import Counter
import numpy as np
import re

# work with data

In [127]:
dataset = load_dataset("oknerazan/english_sentences",streaming=True)

In [128]:
sent = []
wor = []
for ind,i in enumerate(dataset['train']):
    sent.append(re.sub("[^A-za-z0-9 ]",'',i['sentence'].lower()))
    wor.append(i['word'])
    if ind == 1000:
        break

In [129]:
sent[0]

'a simple black spider is a fast face painting design that can make a big impact come halloween'

In [130]:
text = ' '.join([i for i in sent])

In [131]:
alpha = Counter(text)

In [132]:
alpha = list(alpha.keys())

In [133]:
alpha

['a',
 ' ',
 's',
 'i',
 'm',
 'p',
 'l',
 'e',
 'b',
 'c',
 'k',
 'd',
 'r',
 'f',
 't',
 'n',
 'g',
 'h',
 'o',
 'w',
 'u',
 'y',
 'v',
 'x',
 'z',
 'j',
 '1',
 '0',
 'q',
 '2',
 '9',
 '8',
 '4',
 '5',
 '3',
 '6']

In [134]:
# alpha.remove("'")
# alpha.remove('"')


In [135]:
np.random.shuffle(alpha)
# alpha

In [136]:
np.random.randint(1,(10,10))

array([1, 6])

In [137]:
np.random.randint(0,10,10)

array([6, 5, 4, 5, 4, 8, 2, 1, 0, 1])

In [144]:
def corruption(word):
    word2 = word
    step1 = len(word)
    np.random.shuffle(alpha)
    x = np.random.randint(0,step1)
    x = 2
    for i in range(x):
        y = np.random.randint(0,step1)
        # word2[y] = alpha[y]
        k = [*word2]
        k[y] = alpha[y]
        word2 = ''.join([t for t in k])
    return word2

In [145]:
# sent

In [146]:
x

array([ 3, 12,  6, 13,  3,  3])

In [147]:
list(set(x))

[3, 12, 13, 6]

In [227]:
dataset = load_dataset("oknerazan/english_sentences",streaming=True)

sent = []
wor = []
for ind,i in enumerate(dataset['train']):
    sent.append(re.sub("[^A-za-z0-9 ]",'',i['sentence'].lower()))
    wor.append(i['word'])
    if ind == 1000:
        break

text = ' '.join([i for i in sent])
alpha = Counter(text)
alpha = list(alpha.keys())
def corruption(word):
    word2 = word
    step1 = len(word)
    np.random.shuffle(alpha)
    x = np.random.randint(0,step1)
    x = 1
    for i in range(x):
        y = np.random.randint(0,step1)
        # word2[y] = alpha[y]
        k = [*word2]
        k[y] = alpha[y]
        word2 = ''.join([t for t in k])
    return word2
dataset = []
ww = []
corword = []
for i in sent:
    # print(i,j)
    j = i.split()
    c = int(len(j)//5)
    if c ==0:
        c =1
    c = 1
    x = np.random.randint(0,len(j),c)
    for ki in list(set(x)):
        # print(x)
        # x = int(x)
        k = corruption(j[ki])
        corword.append(j[ki])

        i = re.sub(j[ki],k,i)
        ww.append(k)
    # print(i)
    dataset.append(i)
    # break

In [228]:
d = pd.DataFrame()
d['org_text'] = sent
# d['org_word'] = corword
d['wrong_text'] = dataset
# d['wrong_word'] = ww
d.to_csv('/content/drive/MyDrive/roshan_english_data/data_1000_en_alph_v3.csv',index=False)

# work with model

In [229]:
d = pd.read_csv('/content/drive/MyDrive/roshan_english_data/data_1000_en_alph_v3.csv')

In [230]:
sent = d['org_text']

In [231]:
d

,org_text,wrong_text
0,a simple black spider is a fast face painting ...,a simple black spider is a fast face painting ...
1,such fossils are usually fragmentary and conta...,such fossils arv usually fragmentary and conta...
2,a recently discovered fossil group the pterido...,a pecently discovered fossil group the pterido...
3,the moral balance model proposes that most hum...,4he moral balance model proposes that most hum...
4,that fossil down the street,that fossil down the stre4t
...,...,...
305,fossil is a more affordable option than some o...,fossil is a more affordable option than some o...
306,also you can find numerous fossils and petrifi...,also you c3n find numerous fossils and petrifi...
307,look ive never met a fossil that old in my ent...,look iv6 never met a fossil that old in my ent...
308,well not a bad presentation for a fossil,well not a bad presentation ffr a fossil


In [112]:
!pip install evaluate
!pip install jiwer
clear_output()
import evaluate

In [232]:
# Load the WER and CER metrics from the evaluate package
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

In [233]:
predictions = d['wrong_text']
references = d['org_text']

In [234]:
# dataset

In [235]:
# Compute WER
wer_result = wer_metric.compute(predictions=predictions, references=references)
print("WER:", wer_result)

# Compute CER
cer_result = cer_metric.compute(predictions=predictions, references=references)
print("CER:", cer_result)

WER: 0.0941972920696325
CER: 0.016502314348963574


In [177]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
# Set the model to evaluation mode
model.eval()
clear_output()

## feed the model with dataset

In [178]:
# sen = sent[0]
s = []
sentence = 'how aro you doing today'
u = sentence.split()
for ind,i in enumerate(u):
    l = ' '.join([u[c] if ind!=c else '[MASK]' for c in range(len(u))])
    s.append(l)

In [179]:
s

['[MASK] aro you doing today',
 'how [MASK] you doing today',
 'how aro [MASK] doing today',
 'how aro you [MASK] today',
 'how aro you doing [MASK]']

In [180]:
device = 'cpu'

In [181]:
model = model.to(device)

In [182]:
both = []
for i,j in zip(predictions,references):
    both.append(i)
    both.append(j)

In [209]:
u = []
sem = []
ler = []

In [210]:
inputs.input_ids.shape[1]

24

In [211]:
for sentence in both:
    # print(sentence)
    inputs = tokenizer(sentence, return_tensors='pt')
    # print(inputs['input_ids'])
    masked_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
    # print(masked_index)
    # inputs['input_ids'][0][4] = 103
    # Get predictions for the masked token
    with torch.no_grad():
        outputs = model(**inputs.to(device))
    h = torch.softmax(outputs.logits[0], dim=1).max(1)[0]
    sem.append(sentence)
    u.append(round(h[1:-1].mean().item(),4))
    ler.append(inputs.input_ids.shape[1])
    # print('Score for this sentence is: ',round(h[1:-1].mean().item(),4))
    # try:
    #     masked_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
    #     predicted_token_ids = torch.topk(outputs.logits[0, masked_index], k=5).indices
    #     predicted_tokens = [''.join(tok for tok in tokenizer.decode(token_id).split(' ')) for token_id in predicted_token_ids[0]]
    #     print('Best Predicted Word: ',predicted_tokens)
    # except Exception as e:
    #     # print('oops!',e)
    #     pass
    # # for i,j in zip(inputs['input_ids'][0].tolist(),h.tolist()):
    #     # print(f'{tokenizer.decode(i):<15} |  {i:<10} | {j:.4f} |')
    # print(50*'-')

In [212]:
s = pd.DataFrame()
s['sentence'] = sem
s['score'] = u
s['len'] = ler

In [217]:
s

,sentence,score,len
0,a simule b6ack spider is a fast face painting ...,0.9239,24
1,a simple black spider is a fast face painting ...,0.9447,20
2,such qossils are usually qragmentary and conta...,0.8816,17
3,such fossils are usually fragmentary and conta...,0.7715,13
4,a recently discovered dossil group ths pterido...,0.9157,34
...,...,...,...
615,look ive never met a fossil that old in my ent...,0.8415,15
616,well kot a bad presentation for a fossil,0.9248,11
617,well not a bad presentation for a fossil,0.9616,10
618,the stormberg grocp contains south africas ear...,0.7180,15


## find best replacement

In [224]:
np.argmax(forscore)

0

In [ ]:
# hj = []
# for i in predictions[0:1]:
#     predictions[0:1]

In [298]:
Gbest = []
for sentence in predictions:
    Lbest = []
    SLbest = []
    for snd,sentenc in enumerate(sentence.split()):
        kj2 = sentence.split()
        kj2[snd] = '[MASK]'
        sentencer = " ".join([i for i in kj2])
        inputs = tokenizer(sentencer, return_tensors='pt')
        masked_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
        with torch.no_grad():
            outputs = model(**inputs.to(device))
        h = torch.softmax(outputs.logits[0], dim=1).max(1)[0]
        try:
            masked_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
            predicted_token_ids = torch.topk(outputs.logits[0, masked_index], k=5).indices
            predicted_tokens = [''.join(tok for tok in tokenizer.decode(token_id).split(' ')) for token_id in predicted_token_ids[0]]
        except Exception as e:
            pass
        try:
            kj = sentence.split()
            if "##" in predicted_tokens[0]:

                kj2[snd] = ''
                kj2[snd-1] = kj2[snd-1] + predicted_tokens[0][2:]
            else:
                kj2[snd] = predicted_tokens[0]

            sentencer = " ".join([i for i in kj2])
            sentencer = re.sub('  ',' ',sentencer)
            inputs = tokenizer(sentencer, return_tensors='pt')
            with torch.no_grad():
                outputs = model(**inputs.to(device))
            h = torch.softmax(outputs.logits[0], dim=1).max(1)[0]
            sd2 = round(h[1:-1].mean().item(),4)
            inputs = tokenizer(sentence, return_tensors='pt')
            with torch.no_grad():
                outputs = model(**inputs.to(device))
            h = torch.softmax(outputs.logits[0], dim=1).max(1)[0]
            sd1 = round(h[1:-1].mean().item(),4)
            SLbest.append(sd2-sd1)
            Lbest.append(sentencer)
        except Exception as e:
            print('oops!',e)
    Gbest.append(Lbest[np.argmax(SLbest)])

In [300]:
predictions[1]

'such fossils arv usually fragmentary and contain no soft tissue'

In [299]:
Gbest

['a simple black spider is a fast faceless design that can make a big iwpact come halloween',
 'such an arv usually fragmentary and contain no soft tissue']

In [293]:
Gbest = []
for sentence in predictions[3:4]:
    Lbest = []
    SLbest = []
    for snd,sentenc in enumerate(sentence.split()):
        kj2 = sentence.split()
        kj2[snd] = '[MASK]'
        sentencer = " ".join([i for i in kj2])
        # print(sentencer)
        inputs = tokenizer(sentencer, return_tensors='pt')
        masked_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
        with torch.no_grad():
            outputs = model(**inputs.to(device))
        h = torch.softmax(outputs.logits[0], dim=1).max(1)[0]
        try:
            masked_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
            predicted_token_ids = torch.topk(outputs.logits[0, masked_index], k=5).indices
            predicted_tokens = [''.join(tok for tok in tokenizer.decode(token_id).split(' ')) for token_id in predicted_token_ids[0]]
            # print('Best Predicted Word: ',predicted_tokens)
        except Exception as e:
            # print('oops!',e)
            pass
        try:
            kj = sentence.split()
            if "##" in predicted_tokens[0]:

                kj2[snd] = ''
                kj2[snd-1] = kj2[snd-1] + predicted_tokens[0][2:]
                # print(kj2)
            else:
                kj2[snd] = predicted_tokens[0]

            sentencer = " ".join([i for i in kj2])
            # print(sentencer)
            # forsentence.append(sentencer)
            inputs = tokenizer(sentencer, return_tensors='pt')
            with torch.no_grad():
                outputs = model(**inputs.to(device))
            h = torch.softmax(outputs.logits[0], dim=1).max(1)[0]
            sd2 = round(h[1:-1].mean().item(),4)
            # print('Score for this sentence is: ',round(h[1:-1].mean().item(),4))
            inputs = tokenizer(sentence, return_tensors='pt')
            with torch.no_grad():
                outputs = model(**inputs.to(device))
            h = torch.softmax(outputs.logits[0], dim=1).max(1)[0]
            sd1 = round(h[1:-1].mean().item(),4)
            # print('Score for this sentence is: ',round(h[1:-1].mean().item(),4))
            # print(sd2-sd1)
            # p.append(sd2-sd1)
            # print(50*'-')
            SLbest.append(sd2-sd1)
            Lbest.append()
        except Exception as e:
            print('oops!',e)

0

In [240]:
forscores

['a simple black spider is a fast face painting design that can make a big iwpact come [MASK]']

In [263]:
u = 'a simple black spider is a fast face painting design that can make a big impavt on halloween'
# Score for this sentence is:  0.9627
Best_Predicted_Word = ['big', 'little', 'small', 'large', 'super']

inputs = tokenizer(u, return_tensors='pt')
# print(inputs['input_ids'])
masked_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
# print(masked_index)
# inputs['input_ids'][0][4] = 103
# Get predictions for the masked token
with torch.no_grad():
    outputs = model(**inputs.to(device))
h = torch.softmax(outputs.logits[0], dim=1).max(1)[0]
# sem.append(sentencer)
# u.append(round(h[1:-1].mean().item(),4))
# forscore.append(round(h[1:-1].mean().item(),4))
# ler.append(inputs.input_ids.shape[1])
print('Score for this sentence is: ',round(h[1:-1].mean().item(),4))

Score for this sentence is:  0.9704


In [266]:
import torch
from transformers import BertTokenizer, BertForMaskedLM

def find_misspelled_words(sentence):
    # Load pre-trained model tokenizer (vocabulary)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Tokenize input
    tokenized_text = tokenizer.tokenize(sentence)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # Create a tensor of token indices
    tokens_tensor = torch.tensor([indexed_tokens])

    # Load pre-trained model (weights)
    model = BertForMaskedLM.from_pretrained('bert-base-uncased')
    model.eval()

    misspelled_words = []

    with torch.no_grad():
        # Predict all tokens
        outputs = model(tokens_tensor)
        predictions = outputs[0]

        # Iterate over each token in the input sentence
        for i, token in enumerate(tokenized_text):
            # Mask the token we are investigating
            masked_index = i
            tokenized_text_masked = tokenized_text.copy()
            tokenized_text_masked[masked_index] = '[MASK]'
            indexed_tokens_masked = tokenizer.convert_tokens_to_ids(tokenized_text_masked)
            tokens_tensor_masked = torch.tensor([indexed_tokens_masked])

            # Predict the masked token
            outputs_masked = model(tokens_tensor_masked)
            predictions_masked = outputs_masked[0]

            # Get the predicted token
            predicted_index = torch.argmax(predictions_masked[0, masked_index]).item()
            predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

            # Compare the original token with the predicted token
            if token != predicted_token:
                misspelled_words.append((token, predicted_token))

    return misspelled_words

# Example usage
sentence = "Ths is a smple sentence with som misspelled wrds."
u = 'a simple black spider is a fast face painting design that can make a big impavt on halloween'

sentence = u

misspelled_words = find_misspelled_words(sentence)
print("Misspelled words:", misspelled_words)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Misspelled words: [('a', '"'), ('simple', 'the'), ('black', 'the'), ('spider', '"'), ('is', 'with'), ('fast', 'big'), ('face', 'black'), ('painting', '##less'), ('design', ','), ('can', 'will'), ('make', 'be'), ('big', '"'), ('imp', 'big'), ('##av', 'can'), ('##t', '##o'), ('on', '.'), ('halloween', 'the')]


## we want to feed it with one simple sentence

In [53]:
# sen = sent[0]
s = []
sentence = 'how ato you doing today'
sentence = 'you are pretty beautipull .'

u = sentence.split()
for ind,i in enumerate(u):
    l = ' '.join([u[c] if ind!=c else '[MASK]' for c in range(len(u))])
    s.append(l)

In [54]:
for sentence in s:
    print(sentence)
    inputs = tokenizer(sentence, return_tensors='pt')
    # print(inputs['input_ids'])
    masked_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
    # print(masked_index)
    # inputs['input_ids'][0][4] = 103
    # Get predictions for the masked token
    with torch.no_grad():
        outputs = model(**inputs.to(device))
    h = torch.softmax(outputs.logits[0], dim=1).max(1)[0]
    print('Score for this sentence is: ',round(h[1:-1].mean().item(),4))
    try:
        masked_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
        predicted_token_ids = torch.topk(outputs.logits[0, masked_index], k=5).indices
        predicted_tokens = [''.join(tok for tok in tokenizer.decode(token_id).split(' ')) for token_id in predicted_token_ids[0]]
        print('Best Predicted Word: ',predicted_tokens)
    except Exception as e:
        print('oops!',e)
    for i,j in zip(inputs['input_ids'][0].tolist(),h.tolist()):
        print(f'{tokenizer.decode(i):<15} |  {i:<10} | {j:.4f} |')
    print(50*'-')

[MASK] are pretty beautipull .
Score for this sentence is:  0.7673
Best Predicted Word:  ['they', 'you', 'we', 'these', 'both']
[CLS]           |  101        | 0.0313 |
[MASK]          |  103        | 0.6455 |
are             |  2024       | 0.9835 |
pretty          |  3492       | 0.6868 |
beau            |  17935      | 0.9955 |
##tip           |  25101      | 0.9031 |
##ull           |  18083      | 0.1566 |
.               |  1012       | 0.9999 |
[SEP]           |  102        | 0.9347 |
--------------------------------------------------
you [MASK] pretty beautipull .
Score for this sentence is:  0.6861
Best Predicted Word:  ['are', 'look', 'were', 'looked', 'seem']
[CLS]           |  101        | 0.0328 |
you             |  2017       | 0.9725 |
[MASK]          |  103        | 0.3575 |
pretty          |  3492       | 0.8906 |
beau            |  17935      | 0.9906 |
##tip           |  25101      | 0.3926 |
##ull           |  18083      | 0.1988 |
.               |  1012       | 1.

## what if we mask token?

In [25]:
inputs = tokenizer(sentence, return_tensors='pt')
inputs['input_ids'][0]

tensor([ 101, 2129, 2012, 2080, 2017, 2725,  103,  102])

In [182]:
for i in range(inputs['input_ids'].shape[1]-2):
    # print(i)
    # print(sentence)
    inputs = tokenizer(sentence, return_tensors='pt')
    # print(inputs['input_ids'][i])
    inputs['input_ids'][0][i+1] = 103
    # print(inputs['input_ids'][i])
    # print(inputs['input_ids'])
    masked_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
    # print(masked_index)
    # inputs['input_ids'][0][4] = 103
    # Get predictions for the masked token
    with torch.no_grad():
        outputs = model(**inputs.to(device))
    h = torch.softmax(outputs.logits[0], dim=1).max(1)[0]
    print(h[1:-1].mean())
    try:
        masked_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
        predicted_token_ids = torch.topk(outputs.logits[0, masked_index], k=5).indices
        predicted_tokens = [''.join(tok for tok in tokenizer.decode(token_id).split(' ')) for token_id in predicted_token_ids[0]]
        print(predicted_tokens)
    except Exception as e:
        print('oops!',e)
    for i,j in zip(inputs['input_ids'][0].tolist(),h.tolist()):
        print(tokenizer.decode(i),'|',i,'|',round(j,4),'|')
    print(30*'-')

tensor(0.8050, device='cuda:0')
['what', "'", 'how', '"', 'w']
[CLS] | 101 | 0.0323 |
[MASK] | 103 | 0.4621 |
at | 2012 | 0.8865 |
##o | 2080 | 0.5083 |
you | 2017 | 0.9871 |
doing | 2725 | 0.9998 |
today | 2651 | 0.9862 |
[SEP] | 102 | 0.9993 |
------------------------------
tensor(0.7969, device='cuda:0')
['am', 'are', 'vo', 'much', '##mo']
[CLS] | 101 | 0.0278 |
how | 2129 | 0.9993 |
[MASK] | 103 | 0.3276 |
##o | 2080 | 0.4645 |
you | 2017 | 0.9947 |
doing | 2725 | 0.9956 |
today | 2651 | 0.9999 |
[SEP] | 102 | 0.9945 |
------------------------------
tensor(0.9764, device='cuda:0')
['are', 'all', 'what', 'were', 'least']
[CLS] | 101 | 0.0278 |
how | 2129 | 0.9988 |
at | 2012 | 0.95 |
[MASK] | 103 | 0.9316 |
you | 2017 | 0.9961 |
doing | 2725 | 0.9945 |
today | 2651 | 0.9871 |
[SEP] | 102 | 0.9998 |
------------------------------
tensor(0.8000, device='cuda:0')
['is', 'you', 'was', 'are', 'he']
[CLS] | 101 | 0.032 |
how | 2129 | 0.9967 |
at | 2012 | 0.966 |
##o | 2080 | 0.8893 |
[MAS